In [ ]:
%pip install requests

In [4]:
teams = ["TOR"] # Will include all teams later on

In [5]:
url_format = "https://www.basketball-reference.com/teams/{}/2025.html"

In [ ]:
%pip install beautifulsoup4

In [7]:
!python -m pip install certifi


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install pandas
%pip install pyarrow

In [8]:
import certifi
import urllib3
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
)

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO
import re
import os

In [ ]:
# If you get an error, try printing response. If it returns <Reponse [429]>, you are sending too many requests, you have to wait a while to run again
# Check the reponse header to see how long you have to wait before running the req again (we can implement this automatically (optional)), usually 1 hour
# This function will take a while to prevent overflowing the limit (max 30 req /min)

# Since this takes so long, one team might need to be updated on seperate days / and maybe every week worst case

# We use certification to adhere with best practices (this doesn't work on corporate wifi)
# Right now, it sets verification to false, uncomment and replace this line with the one below to use certification

# Initial runtime for the Raptors took 4 min 27.2 seconds 

dfs = []

pattern = r"^(.*?)(?=\s\d{4}-\d{2}|\splayoffs)"

for team in teams:
    url = url_format.format(team)
    time.sleep(2)
    response = requests.get(url, verify=False)
    #response = requests.get(url, verify=certifi.where())
    print(response)
    print(response.headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    roster =  soup.find("table", {"id": "roster"})

    for row in roster.find_all("tr"):
        player_cells = row.find_all('td', {'data-stat': 'player'})

        for cell in player_cells:
            link_tag = cell.find('a')
            if link_tag:
                player_link = link_tag['href']

                full_player_link_format = "https://www.basketball-reference.com{}"
                full_player_link = full_player_link_format.format(player_link)
                time.sleep(2)
                player_response = requests.get(full_player_link, verify=False)
                #player_response = requests.get(full_player_link, verify=certifi.where())
                player_soup = BeautifulSoup(player_response.content, 'html.parser')

                game_logs_header = player_soup.find('p', {'class': 'listhead'}, string="Game Logs")
                    
                if game_logs_header:
                    game_log_links = game_logs_header.find_next_sibling('ul')

                    if game_log_links:
                        for link in game_log_links.find_all('a', href=True):
                            nested_link = link['href']
                            print(f"Nested link: https://www.basketball-reference.com{nested_link}")

                            player_name = player_link.split('/')[-1].replace('.html', '')
                            game_log_link_format = "https://www.basketball-reference.com{}"
                            game_log_link = game_log_link_format.format(nested_link)
                            time.sleep(2)
                            game_log_response = requests.get(game_log_link, verify=False)
                            #game_log_response = requests.get(game_log_link, verify=certifi.where())

                            year = game_log_link.split('/')[-1]

                            if year == "":
                                continue

                            log_soup = BeautifulSoup(game_log_response.content, 'html.parser')

                             # Remove middle headers
                            middle_headers = log_soup.find_all('tr', class_="thead")

                            for header in middle_headers:
                                header.decompose()

                            game_log_table = log_soup.find('table', {'id': 'pgl_basic'})

                            if game_log_table is None:
                                print(f"No table with ID 'pgl_basic' found in nested link: {nested_link}")
                                continue  # Skip this file and move to the next

                            title_tag = log_soup.find('title')

                            if title_tag:
                                title_text = title_tag.get_text()
                                match = re.search(pattern, title_text)

                                if match:
                                    full_name = match.group(1).strip() 

                            game_log = pd.read_html(StringIO(str(game_log_table)))[0]
                        
                            if full_name:
                                game_log["Name"] = full_name

                                # Reorder the columns to put "Name" first
                                columns = ['Name'] + [col for col in game_log.columns if col != 'Name']
                                game_log = game_log[columns]

                            dfs.append(game_log)

In [13]:
game_logs = pd.concat(dfs)
game_logs.drop(['Unnamed: 5', "Rk", "G"], axis=1, inplace=True)
game_logs.rename(columns={'Unnamed: 7': 'Win/Loss'}, inplace=True)

In [14]:
team_names = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

In [15]:
# Have to replace opponent names with full names to merge dataframes in the future based on a common field (opponent)
game_logs['Opp'] = game_logs['Opp'].map(team_names)

In [16]:
game_logs.to_csv("game_logs.csv")

In [17]:
game_logs

,Name,Date,Age,Tm,Opp,Win/Loss,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Gradey Dick,2023-10-25,19-339,TOR,Minnesota Timberwolves,W (+3),0,2:07,0,1,...,0,0,1,0,0,0,1,0,-0.4,+1
1,Gradey Dick,2023-10-27,19-341,TOR,Chicago Bulls,L (-1),0,5:45,0,2,...,1,1,0,0,0,0,0,0,-1.1,-6
2,Gradey Dick,2023-10-28,19-342,TOR,Philadelphia 76ers,L (-7),0,28:27,5,8,...,0,2,1,0,0,1,5,16,11.5,+1
3,Gradey Dick,2023-10-30,19-344,TOR,Portland Trail Blazers,L (-8),0,20:50,2,7,...,5,6,0,2,0,0,3,4,2.9,-9
4,Gradey Dick,2023-11-01,19-346,TOR,Milwaukee Bucks,W (+19),0,21:15,2,10,...,1,1,0,0,0,0,1,5,-1.3,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Kelly Olynyk,2024-04-07,32-354,TOR,Washington Wizards,W (+8),1,34:33,8,12,...,7,9,5,3,1,2,4,21,22.5,+17
79,Kelly Olynyk,2024-04-09,32-356,TOR,Indiana Pacers,L (-17),1,33:48,6,7,...,4,5,4,1,0,1,2,22,23.0,-17
80,Kelly Olynyk,2024-04-10,32-357,TOR,NaN,L (-4),Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
81,Kelly Olynyk,2024-04-12,32-359,TOR,Miami Heat,L (-22),1,24:59,3,7,...,8,11,6,0,0,4,3,8,7.8,-1


In [ ]:
'''
Need to address "Did Not Play", "Did Not Dress", and "Inactive" labels
May need to add a binary feature to indicate whether active (1), or inactive (0)
'''